In [14]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors
import imageio

In [15]:
# Join a week of tracking data to plays with preprocessing criteria
def get_tracking_with_plays(filepath):
    tracking = pd.read_csv(filepath)
    plays = pd.read_csv("plays.csv")
    tracking = tracking[tracking['playId'].isin(tracking[tracking['event'] != 'fumble']['playId'].unique())]
    plays = plays[plays['playNullifiedByPenalty'] == 'N']
    tracking.loc[tracking['playDirection'] == 'left', 'x'] = 120 - tracking.loc[tracking['playDirection'] == 'left', 'x']
    tracking.loc[tracking['playDirection'] == 'left', 'y'] = (160/3) - tracking.loc[tracking['playDirection'] == 'left', 'y']
    tracking.loc[tracking['playDirection'] == 'left', 'dir'] += 180
    tracking.loc[tracking['dir'] > 360, 'dir'] -= 360
    tracking.loc[tracking['playDirection'] == 'left', 'o'] += 180
    tracking.loc[tracking['o'] > 360, 'o'] -= 360
    tracking_with_plays = tracking.merge(plays, on=['gameId', 'playId'], how='left')
    tracking_with_plays['is_on_offense'] = tracking_with_plays['club'] == tracking_with_plays['possessionTeam']
    tracking_with_plays['is_on_defence'] = tracking_with_plays['club'] == tracking_with_plays['defensiveTeam']
    tracking_with_plays['is_ballcarrier'] = tracking_with_plays['ballCarrierId'] == tracking_with_plays['nflId']
    bc_coords=tracking_with_plays.loc[tracking_with_plays['is_ballcarrier']]
    bc_coords['bc_x']=bc_coords['x']
    bc_coords['bc_y']=bc_coords['y']
    bc_coords=bc_coords[['gameId', 'playId', 'frameId', 'bc_x', 'bc_y']]
    tracking_with_plays=tracking_with_plays.merge(bc_coords, on=['gameId', 'playId', 'frameId'], how='left')
    end_frame = tracking_with_plays[tracking_with_plays['event'].isin(['tackle', 'out_of_bounds'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    end_frame.rename(columns={'frameId': 'frameId_end'}, inplace=True)
    start_frame = tracking_with_plays[tracking_with_plays['event'].isin(['run', 'lateral', 'run_pass_option', 'handoff', 'pass_arrived'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    start_frame.rename(columns={'frameId': 'frameId_start'}, inplace=True)
    tracking_with_plays = tracking_with_plays.merge(start_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays.merge(end_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays[(tracking_with_plays['frameId'] <= tracking_with_plays['frameId_end']) &
                                              (tracking_with_plays['frameId'] >= tracking_with_plays['frameId_start'])]
    return tracking_with_plays

In [16]:
# Create figure for football field
def create_football_field():
  # Field rectangle
  rect = patches.Rectangle((0, 0), 53.3, 120, linewidth=0.1, facecolor='#f0f2f6ff', zorder=0)
  fig, ax = plt.subplots(figsize=(3, 7)) # Field size
  ax.add_patch(rect)

  # Field lines
  plt.plot([0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
          [10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80, 80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120], color='grey')

  # Field numbers
  for y in range(20, 110, 10):
    numb = y
    if y > 50:
      numb = 120 - y
    plt.text(5, y-1.5, str(numb - 10), horizontalalignment='center', fontsize=10, color='grey', rotation=270)
    plt.text(53.3-5, y-0.95, str(numb - 10), horizontalalignment='center', fontsize=10, color='grey', rotation=90)

  # Field hashmarks
  for y in range(11,110):
    ax.plot([0.7, 0.4],[y, y], color='grey')
    ax.plot([53.0, 52.5],[y, y], color='grey')
    ax.plot([22.91, 23.57],[y, y], color='grey')
    ax.plot([29.73, 30.39],[y, y],  color='grey')

  # Field end zones
  homeEndzone = patches.Rectangle((0, 0), 53.3, 10, linewidth=0.1, edgecolor='w', facecolor='#f0f2f6ff', zorder=0)
  awayEndzone = patches.Rectangle((0, 110), 53.3, 10, linewidth=0.1, edgecolor='w', facecolor='purple', zorder=0)
  ax.add_patch(homeEndzone)
  ax.add_patch(awayEndzone)
  ax.set_xticks([])
  ax.set_yticks([])
  return fig, ax

In [17]:
# Plot a single frame of tracking data on field
def _forward(x):
    return x
def _inverse(x):
    return x
def plot_frame(play_df, frame_num, nflId, n, description_text):
  # Colour scale with n number of intervals
  colors = ["#f70717","violet","indigo","black","#1f374f","#1E5D9E","#027CFA"]
  cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
  norm = matplotlib.colors.FuncNorm((_forward, _inverse), vmin=-1*n, vmax=n)
  pcm = plt.pcolormesh((np.arange(-1*n, n, 0.01), np.arange(-1*n, n, 0.01)), norm=norm, cmap=cmap, shading='auto')

  frame = play_df[play_df['frameId'] == frame_num]
  fig, ax = create_football_field()

  # Line for place of ball carrier's x at last frame
  finish_line = play_df.loc[(play_df['frameId'] == play_df['frameId_end']) & play_df['bc_x'].notna(), 'bc_x'].iloc[0]
  ax.hlines(y=min(finish_line, 110), xmin=0, xmax=53, colors='black', linestyles='--', linewidth=2, zorder=2)

  # Line for place of ball carrier's predicted end location at each frame
  if frame_num != play_df['frameId_start'].iloc[0]:
    prediction_line = frame.loc[frame['is_ballcarrier'], 'x'].iloc[0] + frame.loc[frame['is_on_defence'], 'Original Prediction'].iloc[0]
    if (prediction_line != finish_line) and (frame_num > ((play_df['frameId_end'].iloc[0]) - 2)):
      prediction_line = finish_line
    ax.hlines(y=min(prediction_line, 110), xmin=0, xmax=53, colors='purple', linestyles='--', linewidth=2, zorder=2)

  # Set field coordinates
  ax.set_ylim([25, 120])

  # Calculate total YSAx per frame and display it in top black rectangle scoreboard style
  Total_YSAx = 0
  Total_YSAx = play_df[(play_df['nflId'] == nflId) & (play_df['frameId'] <= frame_num) & (play_df['NormalizedYSAX'].notna())]['NormalizedYSAX'].sum()
  frame.loc[frame['nflId'] == nflId, 'TotalYSAX'] = Total_YSAx
  TotalYSAx = round(Total_YSAx, 2)
  rect = patches.Rectangle((0, 112), 53, 8, color='black', edgecolor='none', zorder=6)
  ax.add_patch(rect)
  ax.text(53 / 2, 114.7, f"{TotalYSAx}", fontsize=20, fontweight='bold', horizontalalignment='center', color='white', zorder = 6)
  ax.text(53 / 2, 112.3, "YSAx", fontsize=10, horizontalalignment='center', color='white', zorder = 6)

  # Bottom black rectangle play example label
  rect = patches.Rectangle((0, 25), 53, 4, color='black', edgecolor='none', zorder=6)
  ax.add_patch(rect)
  ax.text(53 / 2, 26.2, description_text, fontweight='bold', fontsize=9, horizontalalignment='center', color='white', zorder = 6)

  # Plot ballcarrier, offense, defence, and focused tackler according to colour scale
  # Make focused tackler larger than all dots
  for index, row in frame.iterrows():
      if row['is_ballcarrier']:
        ax.scatter(row['y'], row['x'], color='#5fd102', s=40, zorder=5)
      elif row['is_on_offense']:
        ax.scatter(row['y'], row['x'], color="#faa0a0", s=30, zorder=5)
      elif row['is_on_defence'] and row['nflId'] == nflId:
        ax.scatter(row['y'], row['x'], color=pcm.to_rgba(row['TotalYSAX']), s=80, zorder=4)
      elif row['is_on_defence']:
        ax.scatter(row['y'], row['x'], color="#a0acfa", s=30, zorder=3)

  # Return the frame image
  plt.tight_layout(pad=0)
  fig.canvas.draw()
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
  image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  return image

In [18]:
# Binds togethers plot_frame images for every frame in a given play
def animate(df, gameId, playId, nflId, n, description_text):
  play_df = df[(df['gameId'] == gameId) & (df['playId'] == playId)]
  play_df.fillna(0, inplace=True)
  frames = play_df['frameId'].unique()
  images = [plot_frame(play_df, frame, nflId, n, description_text) for frame in frames]
  # Save images as .gif at 5 frames per second and autoloop
  imageio.mimsave(f"{gameId},{playId},{nflId}.gif", images, fps=5, loop = 0)

In [ ]:
# Get required data for examples 1, 2, 3
tracking4 = get_tracking_with_plays('tracking_week_4.csv')
counterfactual = pd.read_csv('Endzonenegzero-1.csv')
tracking_ysax4 = pd.merge(tracking4, counterfactual, on=['gameId', 'playId', 'frameId', 'nflId'], how='left')
tracking5 = get_tracking_with_plays('tracking_week_5.csv')
tracking_ysax5 = pd.merge(tracking5, counterfactual, on=['gameId', 'playId', 'frameId', 'nflId'], how='left')
tracking9 = get_tracking_with_plays('tracking_week_9.csv')
tracking_ysax9 = pd.merge(tracking9, counterfactual, on=['gameId', 'playId', 'frameId', 'nflId'], how='left')

In [ ]:
# Animate examples 1, 2, 3
animate(tracking_ysax4, 2022100205, 3163, 47871.0, 30, "Example 1: Poor Tackling")
animate(tracking_ysax9, 2022110601, 2063, 43503.0, 5, "Example 2: Average Tackling")
animate(tracking_ysax5, 2022100905, 143, 52445.0, 5, "Example 3: Strong Tackling")